This notebook is used to do a main model run

In [1]:
from epimodel import EpidemiologicalParameters, preprocess_data
from epimodel.pymc3_models.models import ComplexDifferentEffectsWithSeasonalityModel

import numpy as np
import pymc3 as pm

Set Theano Environmental Variables for Parallelisation


In [2]:
data = preprocess_data('../merged_data/data_final_nov.csv', last_day='2020-05-30', smoothing=1)
data.mask_reopenings(print_out = False)

Dropping NPI Mask Wearing
Dropping NPI Travel Screen/Quarantine
Dropping NPI Travel Bans
Dropping NPI Public Transport Limited
Dropping NPI Internal Movement Limited
Dropping NPI Public Information Campaigns
Dropping NPI Symptomatic Testing
Masking invalid values


In [3]:
ep = EpidemiologicalParameters()

In [4]:
bd = ep.get_model_build_dict()

In [5]:
bd

{'gi_mean_mean': 5.06,
 'gi_mean_sd': 0.3265,
 'gi_sd_mean': 2.11,
 'gi_sd_sd': 0.5,
 'deaths_delay_mean_mean': 21.819649695284962,
 'deaths_delay_mean_sd': 1.0056755718977664,
 'deaths_delay_disp_mean': 14.26238141720708,
 'deaths_delay_disp_sd': 5.177442947725441,
 'cases_delay_mean_mean': 10.92830227448381,
 'cases_delay_mean_sd': 0.9387435298564465,
 'cases_delay_disp_mean': 5.406593726647138,
 'cases_delay_disp_sd': 0.2689502951493133}

In [6]:
with ComplexDifferentEffectsWithSeasonalityModel(data) as model:
    model.build_model(seasonality_peak_index=-22, **bd)

In [7]:
with model:
    model.trace = pm.sample(2000, tune=500, cores=4, chains=4, max_treedepth=12, target_accept=0.96)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [DeathsDelayDisp, DeathsDelayMean, InitialSizeDeaths_log, CasesDelayDisp, CasesDelayMean, InitialSizeCases_log, PsiDeaths, PsiCases, GrowthDeathsNoise, GrowthCasesNoise, GrowthNoiseScale, GI_sd, GI_mean, RegionR_noise, HyperRVar, AllCMAlphaNoise, CMAlphaScales, Seasonality, CM_Alpha]


Sampling 4 chains for 500 tune and 2_000 draw iterations (2_000 + 8_000 draws total) took 10077 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [8]:
# save results in a pickle file
import pickle
pickle.dump(model.trace, open('final_final_nov-seasonality_TA096.pkl', 'wb'))